In [1]:
example_1 = """broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a"""

In [12]:
with open('./data/Day 20/input.txt') as input_file:
    input_text = input_file.read()

In [42]:
def parse_input(lines):
    rules = {}
    for line in lines.splitlines():
        line = line.replace(" ","")
        if "->" in line:
            rule, output = line.split("->")
            if "%" in rule or "&" in rule:
                node = rule[1:].strip()
                nodetype= rule[0]
            else:
                node = rule.strip()
                nodetype= "broadcast"
            rules[node] = {"outputs": output.strip().split(","), "type": nodetype}
    return rules

In [151]:
def flipflop(state, input_signal):
    for key in input_signal:
        if input_signal[key]:
            return None, state
        else:
            if state:
                return 0, 0
            else:
                return 1, 1

def conjunction(state, input_signal):
    for key in input_signal:
        if key not in state:
            print(state)
            print(key)
            print("This shouldn't happen")
        state[key] = input_signal[key]
    if all(state.values()):
        return 0, state
    else:
        return 1, state

In [152]:
translate = {"%": flipflop,
 "&": conjunction}

In [153]:
def get_conjunctions(actions):
    conjunctions = []
    for key, value in actions.items():
        if value["type"] == "&":
            conjunctions.append(key)
    return conjunctions

def get_flipflops(actions):
    flipflops = []
    for key, value in actions.items():
        if value["type"] == "%":
            flipflops.append(key)
    return flipflops

In [154]:
def init(actions):
    ### Conjunctions
    conjunctions = get_conjunctions(actions)
    conjunction_states = {}
    for key in conjunctions:
        conjunction_states[key] = {}
        for k, value in actions.items():
            if key in value["outputs"]:
                conjunction_states[key][k] = 0
    ### FlipFlops
    flipflops = get_flipflops(actions)
    flipflop_states = {}
    for key in flipflops:
        flipflop_states[key] = 0
    return {**conjunction_states, **flipflop_states}

In [155]:
def propegate_signal(actions, state, signal):
    new_signal = {}
    for key, value in signal.items():
        if actions[key]["type"] == "broadcast":
            for i in actions[key]["outputs"]:
                new_signal[i] = value
        else:
            for i in actions[key]["outputs"]:
                new_signal[i], state[key] = translate[actions[key]["type"]](
                    state[key], {key:value}
                )
    return new_signal, state

In [156]:
parsed_example = parse_input(example_1)

In [157]:
example_initial_state = init(parsed_example)

In [158]:
initial_state = init(parse_input(input_text))

In [159]:
parsed_example

{'broadcaster': {'outputs': ['a', 'b', 'c'], 'type': 'broadcast'},
 'a': {'outputs': ['b'], 'type': '%'},
 'b': {'outputs': ['c'], 'type': '%'},
 'c': {'outputs': ['inv'], 'type': '%'},
 'inv': {'outputs': ['a'], 'type': '&'}}

In [160]:
signals, state = propegate_signal(
    parsed_example, example_initial_state.copy(), {"a": 0, "b": 0, "c": 0}
)
signals, state = propegate_signal(parsed_example, state, signals)

{'c': 0}
inv
This shouldn't happen


In [162]:
signals

{'c': None, 'inv': None, 'a': 1}